In [1]:
import paho.mqtt.client as PahoMQTT
import time
import pandas as pd
import json

In [2]:
class MyPublisher:
	def __init__(self, clientID):
		self.clientID = clientID
		# create an instance of paho.mqtt.client
		self._paho_mqtt = PahoMQTT.Client(self.clientID, False) 
		# register the callback
		self._paho_mqtt.on_connect = self.myOnConnect
		self._paho_mqtt.on_publish = self.on_publish   
		self.messageBroker = 'localhost'

	def start (self):
		#manage connection to broker
		self._paho_mqtt.connect(self.messageBroker, 1883)
		self._paho_mqtt.loop_start()

	def stop (self):
		self._paho_mqtt.loop_stop()
		self._paho_mqtt.disconnect()

	def myPublish(self, topic, message):
		# publish a message with a certain topic
		self._paho_mqtt.publish(topic, message, 2)

	def myOnConnect (self, paho_mqtt, userdata, flags, rc):
		print ("Connected to %s with result code: %d" % (self.messageBroker, rc))
        
	def on_publish(self, client, userdata, result):             #create function for callback
		print("data published \n")
		pass

In [8]:
filepath = "./Simulation_Output/simulation_data_3years_hourly.csv"
df_columns = ["Date_Time", "t_in", "t_out", "temp_diff", "power_heating", "power_cooling", "power_electricity", "total_power"]
df = pd.read_csv(filepath, sep=',', usecols=df_columns)
df

,Date_Time,t_in,t_out,temp_diff,power_heating,power_cooling,power_electricity,total_power
0,2017/01/01 01:00:00,14.053807,0.925,13.128807,0.000000,0.0,0.610739,0.000000
1,2017/01/01 02:00:00,13.498118,0.650,12.848118,0.000000,0.0,0.610739,0.000000
2,2017/01/01 03:00:00,13.159769,0.850,12.309769,0.221951,0.0,0.610739,0.221951
3,2017/01/01 04:00:00,12.922441,0.975,11.947441,1.160487,0.0,0.610739,1.160487
4,2017/01/01 05:00:00,12.756649,1.000,11.756649,2.092721,0.0,0.610739,2.092721
...,...,...,...,...,...,...,...,...
26275,2019/12/31 20:00:00,16.770866,0.875,15.895866,24.817454,0.0,11.521435,24.817454
26276,2019/12/31 21:00:00,16.874417,0.575,16.299417,24.065884,0.0,11.521435,24.065884
26277,2019/12/31 22:00:00,16.945011,0.275,16.670011,23.390958,0.0,11.521435,23.390958
26278,2019/12/31 23:00:00,16.994231,0.200,16.794231,22.253470,0.0,11.521435,22.253470


In [9]:
df["Date_Time"] = pd.to_datetime(df["Date_Time"] , format='%Y/%m/%d  %H:%M:%S')
# df["Date_Time"] = df["Date_Time"].astype('int64')// 10**9
df = df.set_index("Date_Time")

In [10]:
df

,t_in,t_out,temp_diff,power_heating,power_cooling,power_electricity,total_power
Date_Time,,,,,,,
2017-01-01 01:00:00,14.053807,0.925,13.128807,0.000000,0.0,0.610739,0.000000
2017-01-01 02:00:00,13.498118,0.650,12.848118,0.000000,0.0,0.610739,0.000000
2017-01-01 03:00:00,13.159769,0.850,12.309769,0.221951,0.0,0.610739,0.221951
2017-01-01 04:00:00,12.922441,0.975,11.947441,1.160487,0.0,0.610739,1.160487
2017-01-01 05:00:00,12.756649,1.000,11.756649,2.092721,0.0,0.610739,2.092721
...,...,...,...,...,...,...,...
2019-12-31 20:00:00,16.770866,0.875,15.895866,24.817454,0.0,11.521435,24.817454
2019-12-31 21:00:00,16.874417,0.575,16.299417,24.065884,0.0,11.521435,24.065884
2019-12-31 22:00:00,16.945011,0.275,16.670011,23.390958,0.0,11.521435,23.390958


In [11]:
GATEWAY_NAME = "VirtualBuilding"
publisher = MyPublisher("MyPublisher")
publisher.start()

Connected to localhost with result code: 0
Connected to localhost with result code: 0
Connected to localhost with result code: 0
Connected to localhost with result code: 0


In [12]:
for i in df.index:
    for j in df.loc[i].items():
        key = j[0]
        value = j[1]
        if key == 'total_power':
            measurement = "Power"
        else:
            measurement = "Temperature"
        payload = {
                    "location": str(GATEWAY_NAME),
                    "measurement": measurement,
                    "node": str(key),
                    "time_stamp": str(i),
                    "value": value}
        publisher.myPublish('ict4bd', json.dumps(payload))
        time.sleep(0.1)
test.stop()

Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

dat

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localho

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with r

Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

dat

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with r

...

Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

dat

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with r

Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

dat

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to localhost with result code: 0
Connected to localhost with result code: 0
data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

data published 

Connected to

KeyboardInterrupt: 